### Extract high MQ reads from sorted bam files: ONT reads (basecalled with Albacore2 and Guppy separtelty) & mapped to GDv1.1 

In [268]:
#bam files
WKDIR=/workspace/$USER/github/analysis-workflows/Malus/Red_Flesh_ON
guppy=$WKDIR/Guppy_basecalling/03.minimap2/Guppy_RedFlesh.bam
albacore2=$WKDIR/Red_Flesh_ON/03.minimap2/Redflesh.bam

In [269]:
module load samtools/1.9

#first for albacore2 basecalled reads mapped to GDv1.1 bam file

cd $WKDIR/03.minimap2
samtools view -H $albacore2 > highMQ.sam  #print header
samtools view -F 260 $albacore2 | awk '$5>=40 {print$0}' >> highMQ.sam #select just those mapped reads with MQ>=40
samtools view -S -b -h highMQ.sam > highMQ.bam #convert to bam
samtools depth highMQ.bam > highMQ.cov #compute coverage per bp

#second for Guppy basecalled reads mapped to GDv1.1 bam file

cd $WKDIR/Guppy_basecalling/03.minimap2
samtools view -H $guppy > highMQ.sam  #print header
samtools view -F 260 $guppy | awk '$5>=40 {print$0}' >> highMQ.sam #select just those mapped reads with MQ>=40
samtools view -S -b -h highMQ.sam > highMQ.bam #convert to bam
samtools depth highMQ.bam > highMQ.cov #compute coverage per bp



### Next we use SURVIVOR to cluster the coverage track into a bed file for filtering: https://github.com/fritzsedlazeck/SURVIVOR/wiki







In [270]:
cov_albacore2=$WKDIR/03.minimap2/highMQ.cov
cov_guppy=$WKDIR/Guppy_basecalling/03.minimap2/highMQ.cov

In [ ]:
# Here we allow to cluster regions together that are maximal 100bp away and only consider regions if their coverage is larger then 25!.
# The resulting bed file can be used to filter SV or read file using e.g. bedtools.

In [271]:
cd WKDIR/03.minimap2
/workspace/hrpelg/programs/SURVIVOR-master/Debug/./SURVIVOR bincov $cov_albacore2 0 25 > highMQ.bed

cd $WKDIR/Guppy_basecalling/03.minimap2
/workspace/hrpelg/programs/SURVIVOR-master/Debug/./SURVIVOR bincov $cov_guppy 0 25 > highMQ.bed

### Compare cov files from Albacore2 and Guppy

In [272]:
cov_albacore2_bed=$WKDIR/03.minimap2/highMQ.bed
cov_guppy_bed=$WKDIR/Guppy_basecalling/03.minimap2/highMQ.bed

In [273]:
grep -Fxf $cov_albacore2_bed $cov_guppy_bed | wc -l  #common rows between both file

17984


In [274]:
grep -v -Fxf $cov_albacore2_bed $cov_guppy_bed | wc -l # uniq rows (uniq covered>25 and MQ>40 bp) to Guppy basecalled bam

2062


In [275]:
grep -v -Fxf $cov_albacore2_bed $cov_guppy_bed | grep -v "Chr00" | wc -l  # uniq rows (uniq covered>25 and MQ>40 bp) to Guppy basecalled bam but not in Chr00

945


In [276]:
for i in 00 01 02 03 04 05 06 07 08 09 10 11 12 13 14 15 16 17 # hits in chr09 will be hits still on target but uniq to guppy an not in albacore2
do
cd $WKDIR/Guppy_basecalling/03.minimap2
grep -v -Fxf $cov_albacore2_bed $cov_guppy_bed | grep "Chr$i" > Chr$i'guppy_off_targets.bed'
done

: 1

In [277]:
cd $WKDIR/Guppy_basecalling/03.minimap2
ls -lht Chr* #got files with hits for Chr14 Chr15 Chr16

-rw-rw-r--. 1 hrpelg powerplant    0 Mar 26 14:17 Chr17guppy_off_targets.bed
-rw-rw-r--. 1 hrpelg powerplant 4.3K Mar 26 14:17 Chr16guppy_off_targets.bed
-rw-rw-r--. 1 hrpelg powerplant 2.4K Mar 26 14:17 Chr15guppy_off_targets.bed
-rw-rw-r--. 1 hrpelg powerplant  14K Mar 26 14:17 Chr14guppy_off_targets.bed
-rw-rw-r--. 1 hrpelg powerplant    0 Mar 26 14:17 Chr13guppy_off_targets.bed
-rw-rw-r--. 1 hrpelg powerplant    0 Mar 26 14:17 Chr12guppy_off_targets.bed
-rw-rw-r--. 1 hrpelg powerplant    0 Mar 26 14:17 Chr11guppy_off_targets.bed
-rw-rw-r--. 1 hrpelg powerplant    0 Mar 26 14:17 Chr10guppy_off_targets.bed
-rw-rw-r--. 1 hrpelg powerplant 1.9K Mar 26 14:17 Chr09guppy_off_targets.bed
-rw-rw-r--. 1 hrpelg powerplant    0 Mar 26 14:17 Chr08guppy_off_targets.bed
-rw-rw-r--. 1 hrpelg powerplant    0 Mar 26 14:17 Chr07guppy_off_targets.bed
-rw-rw-r--. 1 hrpelg powerplant    0 Mar 26 14:17 Chr06guppy_off_targets.bed
-rw-rw-r--. 1 hrpelg powerplant    0 Mar 26 14:17 Chr05guppy_off_targets.bed

In [278]:
cd $WKDIR/Guppy_basecalling/03.minimap2
wc -l Chr16guppy_off_targets.bed

182 Chr16guppy_off_targets.bed


In [279]:
cd $WKDIR/Guppy_basecalling/03.minimap2
wc -l Chr14guppy_off_targets.bed

583 Chr14guppy_off_targets.bed


In [280]:
cd $WKDIR/Guppy_basecalling/03.minimap2
wc -l Chr15guppy_off_targets.bed

102 Chr15guppy_off_targets.bed


In [281]:
cd $WKDIR/Guppy_basecalling/03.minimap2
wc -l Chr09guppy_off_targets.bed

78 Chr09guppy_off_targets.bed


In [282]:
# get min max value and min value of each loci in each Chr14

In [283]:
awk 'min=="" || $2 < min {min=$2; minline=$0}; END{ print $1, min}' Chr00guppy_off_targets.bed

Chr00 48263504


In [284]:
awk 'max=="" || $2 > max {max=$2; minline=$0}; END{ print $1, max}' Chr00guppy_off_targets.bed

Chr00 48264621


In [285]:
awk 'min=="" || $2 < min {min=$2; minline=$0}; END{ print $1, min}' Chr14guppy_off_targets.bed

Chr14 30289152


In [286]:
awk 'max=="" || $2 > max {max=$2; minline=$0}; END{ print $1, max}' Chr14guppy_off_targets.bed

Chr14 30291917


In [36]:
# get min max value and min value of each loci in each Chr15

In [287]:
awk 'min=="" || $2 < min {min=$2; minline=$0}; END{ print $1, min}' Chr15guppy_off_targets.bed

Chr15 47766093


In [288]:
awk 'max=="" || $2 > max {max=$2; minline=$0}; END{ print $1, max}' Chr15guppy_off_targets.bed

Chr15 47766888


In [289]:
awk 'min=="" || $2 < min {min=$2; minline=$0}; END{ print $1, min}' Chr16guppy_off_targets.bed

Chr16 38910980


In [290]:
awk 'max=="" || $2 > max {max=$2; minline=$0}; END{ print $1, max}' Chr16guppy_off_targets.bed

Chr16 38919476


In [291]:
# get min max value and min value of each loci in each Chr09  #this are the some bases covered by guppy but not albacore on the targeted locus

In [292]:
awk 'min=="" || $2 < min {min=$2; minline=$0}; END{ print $1, min}' Chr09guppy_off_targets.bed

Chr09 35542756


In [293]:
awk 'max=="" || $2 > max {max=$2; minline=$0}; END{ print $1, max}' Chr09guppy_off_targets.bed

Chr09 35542864


In [294]:
grep -v -Fxf $cov_guppy_bed $cov_albacore2_bed | wc -l # uniq rows (uniq covered>25 and MQ>40 bp) to Albacore2 basecalled bam

55


In [295]:
grep -v -Fxf $cov_guppy_bed $cov_albacore2_bed | grep -v "Chr00" | wc -l # uniq rows (uniq covered>25 and MQ>40 bp) to Albacore2 basecalled bam but not in Chr00

55


In [296]:
for i in 00 01 02 03 04 05 06 07 08 09 10 11 12 13 14 15 16 17 # hits in chr09 will be hits still on target but uniq to guppy an not in Guppy
do
cd $WKDIR/03.minimap2
grep -v -Fxf $cov_guppy_bed $cov_albacore2_bed | grep "Chr$i" > Chr$i'albacore2_off_targets.bed'
done

: 1

In [297]:
ls -lht Chr* #got files with hits for Chr14 Ch15 Chr16

-rw-rw-r--. 1 hrpelg powerplant    0 Mar 26 14:18 Chr17albacore2_off_targets.bed
-rw-rw-r--. 1 hrpelg powerplant    0 Mar 26 14:18 Chr16albacore2_off_targets.bed
-rw-rw-r--. 1 hrpelg powerplant    0 Mar 26 14:18 Chr15albacore2_off_targets.bed
-rw-rw-r--. 1 hrpelg powerplant 1.3K Mar 26 14:18 Chr14albacore2_off_targets.bed
-rw-rw-r--. 1 hrpelg powerplant    0 Mar 26 14:18 Chr13albacore2_off_targets.bed
-rw-rw-r--. 1 hrpelg powerplant    0 Mar 26 14:18 Chr12albacore2_off_targets.bed
-rw-rw-r--. 1 hrpelg powerplant    0 Mar 26 14:18 Chr11albacore2_off_targets.bed
-rw-rw-r--. 1 hrpelg powerplant    0 Mar 26 14:18 Chr10albacore2_off_targets.bed
-rw-rw-r--. 1 hrpelg powerplant    0 Mar 26 14:18 Chr09albacore2_off_targets.bed
-rw-rw-r--. 1 hrpelg powerplant    0 Mar 26 14:18 Chr08albacore2_off_targets.bed
-rw-rw-r--. 1 hrpelg powerplant    0 Mar 26 14:18 Chr07albacore2_off_targets.bed
-rw-rw-r--. 1 hrpelg powerplant    0 Mar 26 14:18 Chr06albacore2_off_targets.bed
-rw-rw-r--. 1 hrpelg powerpl

In [298]:
cd $WKDIR/03.minimap2
wc -l Chr14albacore2_off_targets.bed

55 Chr14albacore2_off_targets.bed


In [300]:
awk 'min=="" || $2 < min {min=$2; minline=$0}; END{ print $1, min}' Chr14albacore2_off_targets.bed

Chr14 30289890


In [301]:
awk 'max=="" || $2 > max {max=$2; minline=$0}; END{ print $1, max}' Chr14albacore2_off_targets.bed

Chr14 30291921


### Look for max coverage on off target loci to add to table off targets (Table 5)

In [302]:
grep "Chr00" $cov_guppy | awk '{if(($2>=48263504)&&($2<=48264621)) print$0}' | awk 'max=="" || $3 > max {max=$3; minline=$0}; END{ print $1, $2, $3}'

Chr00 48264621 28


In [303]:
grep "Chr14" $cov_guppy | awk '{if(($2>=30289152)&&($2<=30291917)) print$0}' | awk 'max=="" || $3 > max {max=$3; minline=$0}; END{ print $1, $2, $3}'

Chr14 30291917 26


In [304]:
grep "Chr15" $cov_guppy | awk '{if(($2>=47766093)&&($2<=47766888)) print$0}' | awk 'max=="" || $3 > max {max=$3; minline=$0}; END{ print $1, $2, $3}'

Chr15 47766888 26


In [306]:
grep "Chr16" $cov_guppy | awk '{if(($2>=38910980)&&($2<=38919476)) print$0}' | awk 'max=="" || $3 > max {max=$3; minline=$0}; END{ print $1, $2, $3}'

Chr16 38919476 41


In [307]:
grep "Chr14" $cov_albacore2 | awk '{if(($2>=30289890)&&($2<=30291921)) print$0}' | awk 'max=="" || $3 > max {max=$3; minline=$0}; END{ print $1, $2, $3}'

Chr14 30291921 26


### Annotation of those 'off-targets' to see if they are some how related to red-flesh locus or has any structural similarity, repeated domains etc

In [156]:
anno=/input/genomic/plant/Malus/Genome/GDDH_v1.1/Decompress/gene_annotation_formatee.gff3
interpro=/input/genomic/plant/Malus/Genome/GDDH_v1.1/Decompress/GDDH13_v1.1_Annotation/Malus_x_domestica_GDDH13_v1.1_interpro.txt
kegg=/input/genomic/plant/Malus/Genome/GDDH_v1.1/Decompress/GDDH13_v1.1_Annotation/Malus_x_domestica_GDDH13_v1.1_KEGG-orthologs.txt
funanno=/input/genomic/plant/Malus/Genome/GDDH_v1.1/Decompress/GDDH13_v1.1_prot.RefSeq69.FuncAnnot.txt

In [127]:
grep "Chr00" $anno | awk '{if(($4>=48263504)&&($5<=48264621)) print$0}' 

Chr00	IRHS-2017	gene	48263538	48264539	.	-	.	ID=gene:MD00G1202200;Name=MD00G1202200;rfam_acc=RF02543;rfam_id=LSU_rRNA_eukarya;Note=LSU_rRNA_eukarya
Chr00	IRHS-2017	ncRNA	48263538	48264539	.	-	.	ID=ncRNA:MD00G1202200;Name=MD00G1202200;Parent=gene:MD00G1202200;rfam_acc=RF02543;rfam_id=LSU_rRNA_eukarya


In [138]:
grep "MD00G1202200" $interpro 

: 1

In [133]:
grep "MD00G1202200" $kegg

: 1

In [151]:
grep "MD00G1202200" $funanno

: 1

In [146]:
grep "Chr14" $anno | awk '{if(($4>=30281000)&&($4<=30292000)) print$0}' # 30289152  30291917

Chr14	IRHS-2017	gene	30282015	30282530	.	+	.	ID=gene:MD14G1221100;Name=MD14G1221100;Note=Oleosin family protein;function=Oleosin family protein
Chr14	IRHS-2017	mRNA	30282015	30282530	.	+	.	ID=mRNA:MD14G1221100;Name=MD14G1221100;Parent=gene:MD14G1221100
Chr14	IRHS-2017	exon	30282015	30282530	.	+	0	ID=exon:MD14G1221100.1;Parent=mRNA:MD14G1221100
Chr14	IRHS-2017	CDS	30282015	30282530	.	+	0	ID=CDS:MD14G1221100.1;Parent=mRNA:MD14G1221100;est_cons=0.0;est_incons=0.0
Chr14	IRHS-2017	gene	30291503	30291994	.	-	.	ID=gene:MD14G1221200;Name=MD14G1221200;Note=Other Eukaryotes  -0 (source: NCBI BLink).;function=Other Eukaryotes  -0 (source: NCBI BLink).
Chr14	IRHS-2017	mRNA	30291503	30291994	.	-	.	ID=mRNA:MD14G1221200;Name=MD14G1221200;Parent=gene:MD14G1221200
Chr14	IRHS-2017	exon	30291503	30291994	.	-	0	ID=exon:MD14G1221200.1;Parent=mRNA:MD14G1221200
Chr14	IRHS-2017	CDS	30291503	30291994	.	-	0	ID=CDS:MD14G1221200.1;Parent=mRNA:MD14G1221200;est_cons=0.0;est_incons=0.0


In [147]:
# So the previous loci is 2351 bp before the gene MD14G1221200 and 6622 bp after of gene MD14G1221100

In [172]:
grep "MD14G1221100" $interpro 

MD14G1221100	IPR000136	GO:0012511;GO:0016021	PF01277	45	157	2.1e-22	T	HMMPfam
MD14G1221100	noIPR	noGo	seg	5	31	NA	?	Seg
MD14G1221100	noIPR	noGo	seg	51	77	NA	?	Seg
MD14G1221100	noIPR	noGo	seg	80	88	NA	?	Seg
MD14G1221100	noIPR	noGo	tmhmm	44	66	NA	?	TMHMM
MD14G1221100	noIPR	noGo	tmhmm	68	90	NA	?	TMHMM
MD14G1221100	noIPR	noGo	tmhmm	95	117	NA	?	TMHMM


In [173]:
grep "MD14G1221100" $kegg

: 1

In [174]:
grep "MD14G1221100" $funanno

MD14G1221100	oleosin 1-like [Pyrus x bretschneideri, Prunus mume, Malus domestica];oleosin 16 kDa-like [Fragaria vesca subsp. vesca];hypothetical protein PRUPE_ppa012323mg [Prunus persica]


In [148]:
grep "MD14G1221200" $interpro 

MD14G1221200	noIPR	noGo	seg	7	16	NA	?	Seg


In [149]:
grep "MD14G1221200" $kegg

: 1

In [152]:
grep "MD14G1221200" $funanno

MD14G1221200	uncharacterized protein LOC103455583 [Malus domestica];uncharacterized protein LOC103929917 [Pyrus x bretschneideri];uncharacterized protein LOC103428315 [Malus domestica]


In [153]:
grep "Chr15" $anno | awk '{if(($4>=47766093)&&($4<=47766888)) print$0}' 

Chr15	IRHS-2017	gene	47766605	47767842	.	-	.	ID=gene:MD15G1386400;Name=MD15G1386400;rfam_acc=RF02543;rfam_id=LSU_rRNA_eukarya;Note=LSU_rRNA_eukarya
Chr15	IRHS-2017	ncRNA	47766605	47767842	.	-	.	ID=ncRNA:MD15G1386400;Name=MD15G1386400;Parent=gene:MD15G1386400;rfam_acc=RF02543;rfam_id=LSU_rRNA_eukarya


In [157]:
grep "MD15G1386400" $interpro 
grep "MD15G1386400" $kegg
grep "MD15G1386400" $funanno

: 1

In [158]:
grep "Chr16" $anno | awk '{if(($4>=38910980)&&($4<=38919476)) print$0}'  

Chr16	IRHS-2017	gene	38910995	38911714	.	+	.	ID=gene:MD16G1282900;Name=MD16G1282900
Chr16	IRHS-2017	mRNA	38910995	38911714	.	+	.	ID=mRNA:MD16G1282900;Name=MD16G1282900;Parent=gene:MD16G1282900
Chr16	IRHS-2017	exon	38910995	38911157	.	+	.	ID=exon:MD16G1282900.1;Parent=mRNA:MD16G1282900
Chr16	IRHS-2017	exon	38911485	38911714	.	+	1	ID=exon:MD16G1282900.2;Parent=mRNA:MD16G1282900
Chr16	IRHS-2017	five_prime_UTR	38910995	38911014	.	+	.	ID=five_prime_UTR:MD16G1282900.0;Parent=mRNA:MD16G1282900;est_cons=60.0;est_incons=0.0
Chr16	IRHS-2017	CDS	38911015	38911157	.	+	0	ID=CDS:MD16G1282900.1;Parent=mRNA:MD16G1282900;est_cons=100.0;est_incons=0.0
Chr16	IRHS-2017	CDS	38911485	38911692	.	+	1	ID=CDS:MD16G1282900.2;Parent=mRNA:MD16G1282900;est_cons=60.6;est_incons=0.5
Chr16	IRHS-2017	three_prime_UTR	38911693	38911714	.	+	.	ID=three_prime_UTR:MD16G1282900.4;Parent=mRNA:MD16G1282900;est_cons=0.0;est_incons=0.0
Chr16	IRHS-2017	gene	38914228	38920342	.	+	.	ID=gene:MD16G1283000;Name=MD16G1283000
Chr16	IRHS-

In [165]:
grep "MD16G1282900" $interpro
grep "MD16G1282900" $kegg
grep "MD16G1282900" $funanno

: 1

In [171]:
grep "MD16G1283000" $interpro
grep "MD16G1283000" $kegg
grep "MD16G1283000" $funanno

: 1

### On target coverage

In [85]:
grep -Fxf $cov_albacore2_bed $cov_guppy_bed | grep "Chr09" | wc -l #on target covered region with cov >25 and at least MQ40 between both files 

7831


In [86]:
grep "Chr09" $cov_albacore2_bed | wc -l 

7831


In [87]:
grep "Chr09" $cov_guppy_bed | wc -l  #Guppy covered a bigger region than albacore2 basecalled mapping

7909


In [22]:
# Conclusion form these last few lines is that albacore2 mapping got a slitly shorted target region covered >25 and MQ>40 than guppy
# The difference is at the beggining of the target, albacore2 starts getting the target well covered later in the sequence


In [91]:
grep "Chr09" $cov_albacore2 | awk '{if($2>=35542701) print$0}'| head -n 165 #

Chr09	35542701	3
Chr09	35542702	3
Chr09	35542703	3
Chr09	35542704	3
Chr09	35542705	3
Chr09	35542706	3
Chr09	35542707	3
Chr09	35542708	3
Chr09	35542709	3
Chr09	35542710	3
Chr09	35542711	3
Chr09	35542712	3
Chr09	35542713	3
Chr09	35542714	3
Chr09	35542715	3
Chr09	35542716	3
Chr09	35542717	3
Chr09	35542718	4
Chr09	35542719	11
Chr09	35542720	15
Chr09	35542721	16
Chr09	35542722	16
Chr09	35542723	16
Chr09	35542724	16
Chr09	35542725	15
Chr09	35542726	13
Chr09	35542727	18
Chr09	35542728	18
Chr09	35542729	17
Chr09	35542730	17
Chr09	35542731	21
Chr09	35542732	21
Chr09	35542733	20
Chr09	35542734	19
Chr09	35542735	18
Chr09	35542736	20
Chr09	35542737	20
Chr09	35542738	22
Chr09	35542739	22
Chr09	35542740	22
Chr09	35542741	21
Chr09	35542742	21
Chr09	35542743	22
Chr09	35542744	22
Chr09	35542745	19
Chr09	35542746	17
Chr09	35542747	21
Chr09	35542748	22
Chr09	35542749	22
Chr09	35542750	21
Chr09	35542751	22
Chr09	35542752	18
Chr09	35542753	18
Chr09	35542754	22
Chr09	35542755	14
Chr09	35542756	22
Chr09	3554

In [38]:
grep "Chr09" $cov_guppy | awk '{if($2>=35542701) print$0}'| head -n 150

Chr09	35542701	3
Chr09	35542702	3
Chr09	35542703	3
Chr09	35542704	3
Chr09	35542705	3
Chr09	35542706	3
Chr09	35542707	3
Chr09	35542708	3
Chr09	35542709	3
Chr09	35542710	3
Chr09	35542711	3
Chr09	35542712	3
Chr09	35542713	3
Chr09	35542714	3
Chr09	35542715	3
Chr09	35542716	3
Chr09	35542717	2
Chr09	35542718	6
Chr09	35542719	13
Chr09	35542720	14
Chr09	35542721	22
Chr09	35542722	22
Chr09	35542723	22
Chr09	35542724	22
Chr09	35542725	23
Chr09	35542726	19
Chr09	35542727	22
Chr09	35542728	23
Chr09	35542729	22
Chr09	35542730	20
Chr09	35542731	24
Chr09	35542732	24
Chr09	35542733	24
Chr09	35542734	23
Chr09	35542735	23
Chr09	35542736	24
Chr09	35542737	24
Chr09	35542738	23
Chr09	35542739	25
Chr09	35542740	25
Chr09	35542741	25
Chr09	35542742	25
Chr09	35542743	25
Chr09	35542744	25
Chr09	35542745	22
Chr09	35542746	22
Chr09	35542747	25
Chr09	35542748	24
Chr09	35542749	23
Chr09	35542750	20
Chr09	35542751	25
Chr09	35542752	22
Chr09	35542753	23
Chr09	35542754	25
Chr09	35542755	21
Chr09	35542756	26
Chr09	3554

In [39]:
# so for albacore2 we get less coverage at the beggining of the target than for guppy 

In [40]:
# But what about at the end of the target 35,551,900

In [92]:
grep "Chr09" $cov_albacore2 | awk '{if($2<=35551900) print$0}'| tail -n 160 #this is the location of digestion of crRNA 4 which didnt work that well 

Chr09	35551741	3
Chr09	35551742	3
Chr09	35551743	3
Chr09	35551744	3
Chr09	35551745	3
Chr09	35551746	3
Chr09	35551747	3
Chr09	35551748	3
Chr09	35551749	3
Chr09	35551750	3
Chr09	35551751	3
Chr09	35551752	3
Chr09	35551753	3
Chr09	35551754	3
Chr09	35551755	3
Chr09	35551756	3
Chr09	35551757	3
Chr09	35551758	3
Chr09	35551759	3
Chr09	35551760	3
Chr09	35551761	3
Chr09	35551762	3
Chr09	35551763	3
Chr09	35551764	2
Chr09	35551765	2
Chr09	35551766	3
Chr09	35551767	3
Chr09	35551768	3
Chr09	35551769	3
Chr09	35551770	2
Chr09	35551771	3
Chr09	35551772	3
Chr09	35551773	3
Chr09	35551774	3
Chr09	35551775	3
Chr09	35551776	3
Chr09	35551777	3
Chr09	35551778	3
Chr09	35551779	3
Chr09	35551780	3
Chr09	35551781	3
Chr09	35551782	3
Chr09	35551783	3
Chr09	35551784	3
Chr09	35551785	3
Chr09	35551786	3
Chr09	35551787	3
Chr09	35551788	3
Chr09	35551789	3
Chr09	35551790	3
Chr09	35551791	3
Chr09	35551792	3
Chr09	35551793	2
Chr09	35551794	3
Chr09	35551795	3
Chr09	35551796	3
Chr09	35551797	3
Chr09	35551798	3
Chr09	35551799

In [51]:
grep "Chr09" $cov_guppy | awk '{if($2<=35551900) print$0}'| tail -n 160 #this is the location of digestion of crRNA 4 which didnt work that well 

Chr09	35551741	3
Chr09	35551742	3
Chr09	35551743	3
Chr09	35551744	3
Chr09	35551745	3
Chr09	35551746	3
Chr09	35551747	3
Chr09	35551748	3
Chr09	35551749	3
Chr09	35551750	3
Chr09	35551751	3
Chr09	35551752	3
Chr09	35551753	3
Chr09	35551754	3
Chr09	35551755	3
Chr09	35551756	3
Chr09	35551757	3
Chr09	35551758	3
Chr09	35551759	3
Chr09	35551760	3
Chr09	35551761	3
Chr09	35551762	3
Chr09	35551763	3
Chr09	35551764	2
Chr09	35551765	2
Chr09	35551766	3
Chr09	35551767	3
Chr09	35551768	3
Chr09	35551769	3
Chr09	35551770	3
Chr09	35551771	3
Chr09	35551772	3
Chr09	35551773	3
Chr09	35551774	3
Chr09	35551775	3
Chr09	35551776	3
Chr09	35551777	3
Chr09	35551778	3
Chr09	35551779	3
Chr09	35551780	3
Chr09	35551781	3
Chr09	35551782	3
Chr09	35551783	3
Chr09	35551784	3
Chr09	35551785	3
Chr09	35551786	3
Chr09	35551787	3
Chr09	35551788	3
Chr09	35551789	3
Chr09	35551790	3
Chr09	35551791	3
Chr09	35551792	3
Chr09	35551793	3
Chr09	35551794	3
Chr09	35551795	3
Chr09	35551796	3
Chr09	35551797	3
Chr09	35551798	3
Chr09	35551799

In [93]:
grep "Chr09" $cov_albacore2 | awk '{if($2<=35550711) print$0}'| tail -n 30  #on crRNA_3

Chr09	35550682	135
Chr09	35550683	131
Chr09	35550684	134
Chr09	35550685	126
Chr09	35550686	127
Chr09	35550687	114
Chr09	35550688	114
Chr09	35550689	111
Chr09	35550690	102
Chr09	35550691	92
Chr09	35550692	88
Chr09	35550693	58
Chr09	35550694	37
Chr09	35550695	3
Chr09	35550696	2
Chr09	35550697	2
Chr09	35550698	2
Chr09	35550699	2
Chr09	35550700	3
Chr09	35550701	3
Chr09	35550702	3
Chr09	35550703	3
Chr09	35550704	3
Chr09	35550705	3
Chr09	35550706	3
Chr09	35550707	3
Chr09	35550708	3
Chr09	35550709	3
Chr09	35550710	3
Chr09	35550711	3


In [56]:
grep "Chr09" $cov_guppy | awk '{if($2<=35550711) print$0}'| tail -n 30  #on crRNA_3

Chr09	35550682	152
Chr09	35550683	148
Chr09	35550684	146
Chr09	35550685	143
Chr09	35550686	144
Chr09	35550687	144
Chr09	35550688	144
Chr09	35550689	141
Chr09	35550690	136
Chr09	35550691	126
Chr09	35550692	121
Chr09	35550693	115
Chr09	35550694	35
Chr09	35550695	2
Chr09	35550696	2
Chr09	35550697	2
Chr09	35550698	3
Chr09	35550699	3
Chr09	35550700	3
Chr09	35550701	3
Chr09	35550702	3
Chr09	35550703	3
Chr09	35550704	3
Chr09	35550705	3
Chr09	35550706	3
Chr09	35550707	3
Chr09	35550708	3
Chr09	35550709	3
Chr09	35550710	3
Chr09	35550711	3


In [ ]:
# so for the  case of crRNA3 (at the end of the target both basecallers works very similarly getting high coverage after 18bp of the cleavage
# However Guppy get the double the coverage at the end of the target than albacore2

## Are off-target randomly distributed or due to similarity to crRNAs? 

In [177]:
#Fasta file with crRNA sequences

In [215]:
crRNAs_query=$WKDIR/crRNA.fa
head $crRNAs_query

>crRNA_RF_1_F
GTCATATCTAAGGACCCGCGTGG
>crRNA_RF_2_F	
TCTGTACTCCGTCTGTCGGTCGG
>crRNA_RF_3_R
AGAAGACTGTCAATCCCGAGTGG
>crRNA_RF_4_F
TGTCTGGAAAGTTTCTAACGCGG



In [216]:
Apple_genome=/input/genomic/plant/Malus/Genome/GDDH_v1.1/Decompress/GDDH13_1-1_formatted.fasta
apple_db=/workspace/hrpelg/Apple_genomes/GDDH13_1-1_formatted_db

In [180]:
#get fasta sequence of 'off-tagets'

In [185]:
module load samtools/1.9
module load bedtools/2.27.1

In [217]:
blast_dir=/workspace/hrpelg/programs/ncbi-blast-2.7.1+/bin

In [183]:
#bed file coordinates of 'off-targets'

In [184]:
off_targets_bed=$WKDIR/off-targets.bed

In [186]:
bedtools getfasta -fi $Apple_genome -bed $off_targets_bed \
-fo $WKDIR/off-targets.fasta

In [232]:
off_targets_fasta=$WKDIR/off-targets.fasta

In [188]:
head $WKDIR/off-targets.fasta

>Chr00:48263504-48264621
AAAGGATGGGCCGGGGAGGAAAGGGGGAGGGACGAATCGAAGCGACGCAGGGCTGAATCTCAGTGGATCGTGGCAGCAAGGCCACTCTGCCACTTACAATACCCCGTCGCGTATTTAAGTCGTCTGCAAAGGATTCTACCCGCCGCTCGGTGGGAATTACGATTCAAGGCGGCCACCGCGGCTCGTCCGCCGCGAGGGCCAGGCCATCGACATGAGCCTTTGGGGGCCGGGGCCCCTACTGCAGGTCGGCAATCGGGCGGCGGGCGCAGGCGTCGCTTCTAGCCCGGATTCTGACTTAGAGGCGTTCAGTCATAATCCAGCGCACGGTAGCTTCGCGCCACTGGCTTTTCAACCAAGCGCGATGACCAATTGTGCGAATCAACGGTTCCTCTCGTACTAGGTTGAATTACTATTGCGACACTGTCATCAGTAGGGTAAAACTAACCTGTCTCACGACGGTCTAAACCCAGCTCACGTTCCCTATTGGTGGGTGAACAATCCAACACTTGGTGAATTCTGCTTCACAATGATAGGAAGAGCCGACATCGAAGGATCAAAAAGCAACGTCGCTATGAACGCTTGGCTGCCACAAGCCAGTTATCCCTGTGGTAACTTTTCTGACACCTCTAGCTTCAAATTCCGAAGGTCTAAAGGATCGATAGGCCACGCTTTCACGGTTCGTATTCGTACTGGAAATCAGAATCAAACGAGCTTTTACCCTTTTGTTCCACACGAGATTTCTGTTCTCGTTGAGCTCATCTTAGGACACCTGCGTTATCTTTTAACAGATGTGCCGCCCCAGCCAAACTCCCCACCTGACAATGTCTTCCGCCCGGATCGGCCCGCGAGCGGGCCTTGGTTCCAAAAAGAGGGGCGATGCCCCGCCTCCGATTCACGGAATAAGTAAAATAACGTTAAAAGTAGTGGTATTTCAATTTCGCCGCGAGGCTCCCACTTATACTACACCTCTCAAGT

In [222]:
blast_dir=/workspace/hrpelg/programs/ncbi-blast-2.7.1+/bin

In [229]:
$blast_dir/blastn -query $crRNAs_query -db $apple_db  \
-outfmt "6 qseqid sseqid pident qcovs length mismatch gapopen qstart qend sstart send qframe sframe evalue bitscore qseq sseq" \
-task blastn-short  \
-out $WKDIR/crRNAs_vs_GDv.1.txt

### Columns of -outfmt 6 results explained http://www.metagenomics.wiki/tools/blast/blastn-output-format-6

supported format specifiers are:

* qseqid    Query Seq-id
* qgi       Query GI
* qacc      Query accesion
* qaccver   Query accesion.version
* qlen      Query sequence length
* sseqid    Subject Seq-id
* sallseqid All subject Seq-id(s), separated by a ';'
* sgi       Subject GI
* sallgi    All subject GIs
* sacc      Subject accession
* saccver   Subject accession.version
* sallacc   All subject accessions
* slen      Subject sequence length
* qstart    Start of alignment in query
* qend      End of alignment in query
* sstart    Start of alignment in subject
* send      End of alignment in subject
* qseq      Aligned part of query sequence
* sseq      Aligned part of subject sequence
* evalue    Expect value
* bitscore  Bit score
* score     Raw score
* length    Alignment length
* pident    Percentage of identical matches
* nident    Number of identical matches
* mismatch  Number of mismatches
* positive  Number of positive-scoring matches
* gapopen   Number of gap openings
* gaps      Total number of gaps
* ppos      Percentage of positive-scoring matches
* frames    Query and subject frames separated by a '/'
* qframe    Query frame
* sframe    Subject frame
* btop      Blast traceback operations (BTOP)
* staxids   Subject Taxonomy ID(s), separated by a ';'
* sscinames Subject Scientific Name(s), separated by a ';'
* scomnames Subject Common Name(s), separated by a ';'
* sblastnames Subject Blast Name(s), separated by a ';'   (in alphabetical order)
* sskingdoms  Subject Super Kingdom(s), separated by a ';'     (in alphabetical order)
* stitle      Subject Title
* salltitles  All Subject Title(s), separated by a '<>'
* sstrand   Subject Strand
* qcovs     Query Coverage Per Subject
* qcovhsp   Query Coverage Per HSP


In [230]:
awk '{print$1,$2,$3,$4,$5,$6,$7,$8,$9,$10,$11,$12,$13,$14,$15,$16,$17,$18}' $WKDIR/crRNAs_vs_GDv.1.txt

crRNA_RF_1_F Chr09 100.000 100 23 0 0 1 23 35542701 35542723 1 1 6.65e-05 46.1 GTCATATCTAAGGACCCGCGTGG GTCATATCTAAGGACCCGCGTGG 
crRNA_RF_2_F Chr09 100.000 100 23 0 0 1 23 35542848 35542870 1 1 6.65e-05 46.1 TCTGTACTCCGTCTGTCGGTCGG TCTGTACTCCGTCTGTCGGTCGG 
crRNA_RF_3_R Chr09 100.000 100 23 0 0 1 23 35550711 35550689 1 -1 6.65e-05 46.1 AGAAGACTGTCAATCCCGAGTGG AGAAGACTGTCAATCCCGAGTGG 
crRNA_RF_3_R Chr14 94.737 83 19 1 0 1 19 5006898 5006880 1 -1 3.9 30.2 AGAAGACTGTCAATCCCGA AGAACACTGTCAATCCCGA 
crRNA_RF_4_F Chr09 100.000 100 23 0 0 1 23 35551878 35551900 1 1 6.65e-05 46.1 TGTCTGGAAAGTTTCTAACGCGG TGTCTGGAAAGTTTCTAACGCGG 
crRNA_RF_4_F Chr06 100.000 65 15 0 0 3 17 23658580 23658566 1 -1 3.9 30.2 TCTGGAAAGTTTCTA TCTGGAAAGTTTCTA 
crRNA_RF_4_F Chr17 100.000 70 15 0 0 2 16 9145377 9145363 1 -1 3.9 30.2 GTCTGGAAAGTTTCT GTCTGGAAAGTTTCT 
crRNA_RF_4_F Chr17 100.000 70 15 0 0 3 17 16542983 16542969 1 -1 3.9 30.2 TCTGGAAAGTTTCTA TCTGGAAAGTTTCTA 


In [231]:
 module load mafft/7.307

In [236]:
cd $WKDIR
mafft  --localpair --maxiterate 100 $off_targets_fasta > $WKDIR/mafft_off_target_output.txt


generating a scoring matrix for nucleotide (dist=200) ... done
All-to-all alignment.
tbfast-pair (nuc) Version 7.307 alg=L, model=DNA200 (2), 2.00 (6.00), -0.10 (-0.30), noshift, amax=0.0
0 thread(s)

Loading 'hat3.seed' ... 
done.
Writing hat3 for iterative refinement
generating a scoring matrix for nucleotide (dist=200) ... done
Gap Penalty = -1.53, +0.00, +0.00
treein = 0
compacttree = 0
Constructing a UPGMA tree ... 
    0 / 4
done.

Progressive alignment ... 
STEP     3 /3 c
done.
tbfast (nuc) Version 7.307 alg=A, model=DNA200 (2), 1.53 (4.59), -0.00 (-0.00), noshift, amax=0.0
0 thread(s)

minimumweight = 0.000010
autosubalignment = 0.000000
nthread = 0
randomseed = 0
blosum 62 / kimura 200
poffset = 0
niter = 16
sueff_global = 0.100000
Loading 'hat3' ... done.
generating a scoring matrix for nucleotide (dist=200) ... done

    0 / 4
Segment   1/  1    1-8733
STEP 006-002-1  rejected..   
Converged.

done
dvtditr (nuc) Version 7.307 alg=A, model=DNA200 (2), 1.53 (4.59), -0.00 (-0

In [238]:
tail $WKDIR/mafft_off_target_output.txt

ggctggaagagcaccgcacgtcgcgcggtgtccggtgcgcccccggcggcccttgaaaat
ccggaggaccgagtaccgttcgcgcccggtcgtactcataaccgcatcaggtctccaagg
tgaacagcctctggtcgatggaacaatgtaggcaagggaagtcggcaaaatggatccgta
acttcgggaaaaggattggctctgaggaacgggcacgggggtcccggcccggaacccgtc
ggctgtcggcgaactgctcgagctgctcccgcggcgagagcgggccgccgcgtgccggcc
gggggaaggaccgggaacggtcccctcgggggccttccccgggcagcgaacgttcagctc
agaactggtacggacaaggggaatccgactgtttaattaaaacaaagcattgcgatggtc
cctgcggatgctaacgcaatgtgatttctgcccagtgctctgaatgtcaaagtgaagaaa
ttcaaccaagcgcgggtaaacggcgggagtaactatgactctcttaaggtagccaaatgc
ctcgtcatctaattagtgacgcgcatgaatggattaacgagattcccac


 Removed form paper but kept here just in case 
 
 The multiple sequencing alignment between these four ‘off-target’ regions showed a shared region of 320 bp between the two ‘off-target’ regions found on chromosomes 0 and 15 
 
Not sure if this is relevant. What is exactly in Chr00? Reads that couldn’t be assembled, can be also multi-mapping reads??

Both regions are in genes which their annotation is:

Chr15 (795bp)
ID=gene:MD15G1386400;Name=MD15G1386400;rfam_acc=RF02543;rfam_id=LSU_rRNA_eukarya;Note=LSU_rRNA_eu

Chr00 (1117bp)
ID=gene:MD00G1202200;Name=MD00G1202200;rfam_acc=RF02543;rfam_id=LSU_rRNA_eukarya;Note=LSU_rRNA_eukarya

18 S Ribosomal RNA genes

Shared region is 320bp

If we include this, I need to add in method part the Geneious MSA alignment min identity 70% and maybe add a supplemental fig of the visual alignment

I also did alignment using mafft in Linux but I would need to install or see how I visualize the output generated, so I decided go to Geneious instead.
